<a href="https://colab.research.google.com/github/ipeirotis/sql_autograding/blob/main/finetune_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U openai
!pip install -q google-cloud-secret-manager
!pip install -U PyMySQL sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00


In [ ]:
import gcsfs
import pandas as pd
import io
import openai
import os

from google.cloud import secretmanager

from google.colab import auth

import requests

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split

import json

from openai import OpenAI

In [ ]:
# Login using the account that has access to the Google project
# in order to access the resources for the project
auth.authenticate_user()

In [ ]:
def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload of the given secret version and return it.

    Args:
        project_id (str): Google Cloud project ID.
        secret_id (str): ID of the secret to access.
        version_id (str): ID of the version to access.
    Returns:
        str: The secret version's payload, or None if
        the version does not exist.
    """
    try:
        client = secretmanager.SecretManagerServiceClient()
        name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
        response = client.access_secret_version(request={"name": name})
        return response.payload.data.decode("UTF-8")
    except Exception as e:
        print(f"Failed to access the secret version: {e}")
        return None


openai_key = access_secret_version("sql-autograding", "openai-gpt4-32k", "latest")
openai.api_key = openai_key
client = OpenAI(
  api_key=openai_key,  # this is also the default, it can be omitted
)

In [ ]:
URL = "https://api.openai.com/v1/chat/completions"

In [ ]:
# read the dataset
fs = gcsfs.GCSFileSystem(project="sql_autograding")
# open a file with GPT 4 grade & feedback
with fs.open("gs://sql_autograding/few_shots_cleaned_final.csv") as f:
    data = pd.read_csv(f)

In [ ]:
data = data.drop(columns=['Unnamed: 0', '4_grade','3_few_mp', '3_answer', '3_grade', '3_feedback'])
data.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,4_few_mp,num,4_answer,4_feedback,4_grade_scaled
0,63,"SELECT FirstName, LastName, HireDate FROM Emp...",amc10323,5.0,Final Exam,"We would like to see just the FirstName, LastN...",northwind,"SELECT FirstName, LastName, HireDate FROM Empl...","[{'Categories': ['CategoryID', 'CategoryName',...",5.0,\n a. We are working with the northwind d...,2437,Grade: 10/10\n\nFeedback: Excellent job! Your ...,Excellent job! Your SQL query is correct and i...,5.0
1,16,"select T.track_name, A.artist_name, AL.album_n...",jd2855,0.0,Assignment 3: Joins,"List all the track names, the corresponding al...",music,"SELECT track_name, artist_name, album_name FRO...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2730,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,Great job! Your SQL query correctly joins the ...,1.0
2,3,SELECT * FROM album;,mc10018,0.0,Assignment 2B: Selection Queries,Show all the albums.,music,SELECT * FROM album,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2091,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,Great job! Your SQL query is correct and it wi...,1.0
3,0,Select artist_name from artist order by arti...,lka2053,0.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2519,Grade: 10/10\n\nFeedback: Your answer is corre...,Your answer is correct. You have successfully ...,1.0
4,24,"use music; SELECT A.artist_id, COUNT(B.album_...",sf4210,0.0,Assignment 4: Aggregations,"For each artist, count the number of albums in...",music,"SELECT artist_id, COUNT(album_id) AS cnt_album...","[{'album': ['artist_id', 'album_id', 'album_na...",11.0,\n a. We are working with the music datab...,2456,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,Great job! Your SQL query correctly counts the...,11.0


### Train-test split

In [ ]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=1234)
len(train_df), len(test_df)

(320, 80)

### Get correct grade + feedback from GPT 4 for training set

In [ ]:
train_df['prompt'] = "In database " + train_df['Database'].astype(str) + ", we ask the question: " + train_df['Question'] + " The student answered " + train_df['SubmittedAnswer'] + " The correct answer was " + train_df['AnswerKey']
# change TA grade to GPT4 grade
train_df['message'] = "The GPT4 grader assigned the grade " + train_df['4_grade_scaled'].astype(str) + " points out of the full score of " + train_df['full_score'].astype(str) + " points. The feedback is:" + train_df['4_feedback']

train_df.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,4_few_mp,num,4_answer,4_feedback,4_grade_scaled,prompt,message
33,51,"SELECT state, COUNT(origin), COUNT(carrier), S...",aah9125,7.5,Final Exam,Flights: For each state of the origin airport ...,flights,"SELECT state , COUNT(DISTINCT T.origin) AS air...","[{'airports': ['airport', 'state', 'state_name...",7.5,\n a. We are working with the flights dat...,2425,Grade: 10\n\nFeedback: Excellent job! Your SQL...,Excellent job! Your SQL query correctly calcul...,7.5,"In database flights, we ask the question: Flig...",The GPT4 grader assigned the grade 7.5 points ...
253,4,SELECT * FROM artist;,pt2396,1.0,Assignment 2B: Selection Queries,Show all the artists.,music,SELECT * FROM artist,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2183,Grade: 10/10\n\nFeedback: Your answer is corre...,Your answer is correct. You have successfully ...,1.0,"In database music, we ask the question: Show a...",The GPT4 grader assigned the grade 1.0 points ...
118,38,CREATE TEMPORARY TABLE male_actors select * fr...,ve418,1.0,Assignment 5: Subqueries,Find the eligible names (see definition of eli...,imdb,CREATE TEMPORARY TABLE female_names AS SELECT ...,"[{'actors': ['id', 'first_name', 'last_name', ...",1.0,\n a. We are working with the imdb databa...,2637,Grade: 7/10\n\nFeedback: The student's answer ...,The student's answer is partially correct. The...,0.7,"In database imdb, we ask the question: Find th...",The GPT4 grader assigned the grade 0.7 points ...
300,7,SELECT * FROM album WHERE album_name > 'E%...,ahz2022,0.5,Assignment 2C: Filtering Queries,Find all albums with a title that begins with ...,music,SELECT * FROM album WHERE album_name >= 'F' AN...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2302,Grade: 7/10\n\nFeedback: Your SQL query is alm...,"Your SQL query is almost correct, but you have...",0.7,"In database music, we ask the question: Find a...",The GPT4 grader assigned the grade 0.7 points ...
115,0,show databases; USE music; SHOW Tables; SEL...,soj206,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2469,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,Great job! Your SQL query is correct and it su...,1.0,"In database music, we ask the question: List a...",The GPT4 grader assigned the grade 1.0 points ...


In [ ]:
test_df['prompt'] = "In database " + test_df['Database'].astype(str) + ", we ask the question: " + test_df['Question'] + " The student answered " + test_df['SubmittedAnswer'] + " The correct answer was " + test_df['AnswerKey'] + " Please grade this question on a scale base on the full score " + test_df['full_score'].astype(str) + " and provide feedback explaining why you gave this grade."
test_df['message'] = "The GPT4 grader assigned the grade " + test_df['4_grade_scaled'].astype(str) + " points out of the full score of " + test_df['full_score'].astype(str) + " points. The feedback is:" + test_df['4_feedback']
test_df.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,4_few_mp,num,4_answer,4_feedback,4_grade_scaled,prompt,message
44,27,"USE music; SHOW tables; SELECT artist_id, al...",soj206,0.8,Assignment 4: Aggregations,"List the IDs of the albums, where the total al...",music,"SELECT artist_id, album_id, SUM(time) AS total...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2427,Grade: 7/10\n\nFeedback: Your SQL query is mos...,"Your SQL query is mostly correct, but you miss...",0.7,"In database music, we ask the question: List t...",The GPT4 grader assigned the grade 0.7 points ...
110,9,SELECT * FROM track WHERE artist_id = 3 and ...,sl11045,0.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2252,Grade: 10/10\n\nFeedback: Excellent job! Your ...,Excellent job! Your SQL query is correct and i...,1.0,"In database music, we ask the question: Find t...",The GPT4 grader assigned the grade 1.0 points ...
78,12,"SELECT * FROM artist WHERE artist_name = ""Th...",yl12110,1.0,Assignment 2C: Filtering Queries,Show the entry for the artist named The Rollin...,music,SELECT * FROM artist WHERE artist_name = 'The ...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2174,Grade: 10/10\n\nFeedback: Excellent job! Your ...,Excellent job! Your SQL query is correct and i...,1.0,"In database music, we ask the question: Show t...",The GPT4 grader assigned the grade 1.0 points ...
181,54,"SELECT C.CustomerID, C.CompanyName, SUM(IFNUL...",ky2449,10.0,Final Exam,"For each customer, we want to calculate how mu...",northwind,"SELECT C.CustomerID, C.CompanyName , COALESCE(...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0,\n a. We are working with the northwind d...,2537,Grade: 10/10\n\nFeedback: Excellent job! Your ...,Excellent job! Your SQL query correctly calcul...,10.0,"In database northwind, we ask the question: Fo...",The GPT4 grader assigned the grade 10.0 points...
285,38,CREATE TEMPORARY TABLE male_names AS SELECT f...,qz2313,1.0,Assignment 5: Subqueries,Find the eligible names (see definition of eli...,imdb,CREATE TEMPORARY TABLE female_names AS SELECT ...,"[{'actors': ['id', 'first_name', 'last_name', ...",1.0,\n a. We are working with the imdb databa...,2070,Grade: 9/10\n\nFeedback: Your answer is almost...,Your answer is almost correct. You have correc...,0.9,"In database imdb, we ask the question: Find th...",The GPT4 grader assigned the grade 0.9 points ...


In [ ]:
# filter only prompt & message cols and submit training df
df_train_ft = train_df[['prompt', 'message']]
df_train_ft.head()

,prompt,message
33,"In database flights, we ask the question: Flig...",The GPT4 grader assigned the grade 7.5 points ...
253,"In database music, we ask the question: Show a...",The GPT4 grader assigned the grade 1.0 points ...
118,"In database imdb, we ask the question: Find th...",The GPT4 grader assigned the grade 0.7 points ...
300,"In database music, we ask the question: Find a...",The GPT4 grader assigned the grade 0.7 points ...
115,"In database music, we ask the question: List a...",The GPT4 grader assigned the grade 1.0 points ...


In [ ]:
def create_jsonl_format(row):
    return {
        "messages": [
            {"role": "system", "content": "You are a SQL grader that takes in questions and gives correct grades and feedback"},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['message']}
        ]
    }

In [ ]:
# Write the list of dictionaries to a JSONL file
# Apply the function to each row and convert the DataFrame to a list of dictionaries
jsonl_data = df_train_ft.apply(create_jsonl_format, axis=1).tolist()

with open('training_set.jsonl', 'w') as file:
    for item in jsonl_data:
        file.write(json.dumps(item) + '\n')

### Plug in training set to fine-tune model (davinci >> vanilla)?

In [ ]:
# df_train_ft.to_json('training_data.jsonl', orient='records', lines=True)

In [ ]:
# !openai tools fine_tunes.prepare_data -f training_data.jsonl -q

In [ ]:
os.environ["OPENAI_API_KEY"] = openai_key

### Test testing set and evaluate result

In [ ]:
df_test_ft = test_df[['prompt', 'message']]

In [ ]:
df_test_ft.head()

,prompt,message
44,"In database music, we ask the question: List t...",The GPT4 grader assigned the grade 0.7 points ...
110,"In database music, we ask the question: Find t...",The GPT4 grader assigned the grade 1.0 points ...
78,"In database music, we ask the question: Show t...",The GPT4 grader assigned the grade 1.0 points ...
181,"In database northwind, we ask the question: Fo...",The GPT4 grader assigned the grade 10.0 points...
285,"In database imdb, we ask the question: Find th...",The GPT4 grader assigned the grade 0.9 points ...


In [ ]:
jsonl_data = df_test_ft.apply(create_jsonl_format, axis=1).tolist()

with open('testing_set.jsonl', 'w') as file:
    for item in jsonl_data:
        file.write(json.dumps(item) + '\n')

In [ ]:
# df_test_ft.to_json('testing_data.jsonl', orient='records', lines=True)

In [ ]:
# !openai tools fine_tunes.prepare_data -f testing_data.jsonl -q

In [ ]:
client.files.create(
  file=open("/content/training_set.jsonl", "rb"),
  purpose='fine-tune'
)

FileObject(id='file-UjKnrzA2cT1yATXKWqaq2XxN', bytes=378715, created_at=1700987887, filename='training_set.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.files.create(
  file=open("testing_set.jsonl", "rb"),
  purpose='fine-tune'
)

FileObject(id='file-8eFaQZ3roII4RNxfluzfgfGH', bytes=112433, created_at=1700987890, filename='testing_set.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
# openai.File.create(
#   file=open("training_data_prepared.jsonl", "rb"),
#   purpose='fine-tune'
# )

<File file id=file-WvZYP7b3fsuRR2R0jKxZLjqV at 0x79e3ae26f9c0> JSON: {
  "object": "file",
  "id": "file-WvZYP7b3fsuRR2R0jKxZLjqV",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 230498,
  "created_at": 1695674850,
  "status": "uploaded",
  "status_details": null
}

In [ ]:
# openai.FineTuningJob.create(training_file="file-WvZYP7b3fsuRR2R0jKxZLjqV", model="gpt-3.5-turbo")

In [ ]:
# !openai api fine_tunes.results -i ft-qPmQWstQ3R8R5KNanJGuHstU > result.csv

In [ ]:
# results = pd.read_csv('result.csv')
# results.head()

,step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy
0,1,505,1,0.442257,0.0,0.469880,0.834735,0.0,0.590909
1,2,810,2,0.501740,0.0,0.550000,NaN,NaN,NaN
2,3,915,3,0.973854,0.0,0.428571,NaN,NaN,NaN
3,4,1060,4,0.839081,0.0,0.410256,NaN,NaN,NaN
4,5,1197,5,0.951844,0.0,0.547170,NaN,NaN,NaN


In [ ]:
response = client.fine_tuning.jobs.create(training_file="file-UjKnrzA2cT1yATXKWqaq2XxN", validation_file = "file-8eFaQZ3roII4RNxfluzfgfGH", model="gpt-3.5-turbo")

print(response)

FineTuningJob(id='ftjob-onCL9VuF1XGIYe4dFPA1ijSY', created_at=1700987935, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-nuVeAYmIh1ClQf5P4Yu4xYZw', result_files=[], status='validating_files', trained_tokens=None, training_file='file-UjKnrzA2cT1yATXKWqaq2XxN', validation_file='file-8eFaQZ3roII4RNxfluzfgfGH')
